In [2]:
import syntok.segmenter as segmenter
import os
import re
import gender_guesser.detector as ggd
import gender_detector as g_d
import nltk as nltk
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.corpus import stopwords

import difflib
import spacy
from collections import Counter
import en_core_web_sm
nlp = spacy.load("en_core_web_md")
nlp.max_length = 2000000

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/tommasobattisti/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/tommasobattisti/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:
directory_paths = ["Corpus/female-writers/", "Corpus/male-writers/"]
file_paths = []

for path in directory_paths:
    for root, directories, files in os.walk(path):
        for filename in files:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)



file_paths_1880 = []
for root, directories, files in os.walk("Corpus1880/"):
    for filename in files:
        # Join the two strings in order to form the full filepath.
        filepath = os.path.join(root, filename)
        file_paths_1880.append(filepath)

file_paths

['Corpus/female-writers/1872_de-la-ramee-a-dog-of-flanders.txt',
 'Corpus/female-writers/1857_browne-grannys-wonderful-chair.txt',
 'Corpus/female-writers/1877_molesworth-the-cuckoo-clock.txt',
 'Corpus/female-writers/1877_sewell-black-beauty.txt',
 'Corpus/female-writers/1902_potter-the-tale-of-peter-rabbit.txt',
 'Corpus/female-writers/1899_nesbit-the-story-of-the-treasure-seekers.txt',
 'Corpus/female-writers/1869_ewing-mrs-overtheways-remembrances.txt',
 'Corpus/female-writers/1886_hodgson-burnett-little-lord-fauntleroy.txt',
 'Corpus/male-writers/1888_wilde-the-happy-prince-and-other-tales.txt',
 'Corpus/male-writers/1871_macdonald-at-the-back-of-the-north-wind.txt',
 'Corpus/male-writers/1894_kipling-the-jungle-book.txt',
 'Corpus/male-writers/1869_dickens-david-copperfield.txt',
 'Corpus/male-writers/1865_carroll-alices-adventures-in-wonderland.txt',
 'Corpus/male-writers/1857_hughes-tom-browns-school-days.txt',
 'Corpus/male-writers/1883_stevenson-treasure-island.txt',
 'Corpus

In [ ]:
def get_characters(book, book_path):
    names_list = []

    #this following commented part was useful to manage only the specific case of "The story of the treasure seekers" by Nesbit
    """ if re.findall("nesbit-the-story-of-the-treasure-seekers\.txt$", book_path):
        book = re.sub('([A-Z]{2,})',lambda low: low.group(1).lower() , book)
        book = re.sub('H\.\sO\.', 'Horace Octavius', book)
        book = re.sub('Horace\sOctavius(\n)+', 'Horace Octavius.', book)  
        book = re.sub('Horace\sOctavius\s([A-Z])', lambda pat: "Horace Octavius " + pat.group(1).lower(), book)
        book = re.sub('([A-Z]\w+)\sHorace\sOctavius', lambda pat: pat.group(1).lower() + " Horace Octavius", book)
        book = re.sub('G\.\sB\.', 'Generous Benefactor', book) """

    book = re.sub('‘([A-Z])', lambda pat: "‘" + pat.group(1).lower(), book)
    book = re.sub('\n', ' ', book)
    book = re.sub("’[^st]", ' ', book)
    book = re.sub("‘", " ", book)
    book = re.sub('--', ' ', book)
    book = re.sub('-', ' ', book)
    book = re.sub('_', ' ', book)
    book = re.sub('- -', ' ', book)
    book = re.sub('\*', ' ', book)
    book = re.sub('\s+', ' ', book)

    
    nltk_name_dict, sentences_num = get_charaters_nltk(book)
    spacy_name_dict = get_characters_spacy(book)

    nltk_name_dict = rebalance_name_dict(nltk_name_dict, sentences_num)
    spacy_name_dict = rebalance_name_dict(spacy_name_dict, sentences_num)

    print("nltk\n", nltk_name_dict)
    print("spacy\n", spacy_name_dict)

    for key in spacy_name_dict:
        if sentences_num >= 17000:
            if spacy_name_dict[key] >= 35 or key in nltk_name_dict and nltk_name_dict[key] >= 20 and key not in names_list:
                names_list.append(key)
        else:
            if spacy_name_dict[key] >= 7 or key in nltk_name_dict and nltk_name_dict[key] >= 2 and key not in names_list:
                names_list.append(key)
    
    for key in nltk_name_dict:
        if sentences_num >= 17000:
            if key not in names_list and nltk_name_dict[key] >= 35:
                names_list.append(key)
        else:
            if key not in names_list and nltk_name_dict[key] >= 7:
                names_list.append(key)
    
    return check_name_strings(names_list)


def check_name_strings(names_list):
    regex_dict = {".+!":("!", ""), "[Tt]he\s.+":("[Tt]he\s", ""), "[oO]\s.+": ("[Oo]\s", ""), ".+['’]s": ("['’]s", ""), ".+\s['’]s": ("\s['’]s", ""), "[Pp]rince\s.+": ("[Pp]rince\s", ""), "[Pp]rincess\s.+": ("[Pp]rincess\s", ""), "[Kk]ing\s.+": ("[Kk]ing\s", ""), "[Qq]ueen\s.+": ("[Qq]ueen\s", ""), "[Dd]ame\s.+": ("[Dd]ame\s", ""), "[Ll]ady\s.+": ("[Ll]ady\s", ""), ".+['’,.]": ("['’,.]", "")}
    name_set = set()
    not_names = ["sir", "lord", "lady", "king", "queen", "prince", "princess", "dame", "miss", "mister", "mr", "cap", "i", "me", "you", "aunt", "mother", "father", "uncle"]
    for name in names_list:
        s_name = name.strip()
        if s_name.lower() not in not_names and not re.match("[Pp]art\s.+|[Cc]hapter\s.+", s_name):
            for regex in regex_dict:
                if re.match(regex, name):
                    s_name = re.sub(regex_dict[regex][0], regex_dict[regex][1], s_name)
            if s_name[0].isupper() and not (re.match("Christmas", s_name) or re.match(".+\sChristmas", s_name) or re.match(".+\s[Ii]sland", s_name) or re.match(".+\s[Ll]ake", s_name)):
                name_set.add(s_name.strip())
    return list(name_set)



def rebalance_name_dict(name_dict, sentences_number):
    names_list = list(name_dict.keys())
    while len(names_list) > 0:
        last_nl = names_list.pop()
        for name in names_list:
            similarity = difflib.SequenceMatcher(None, last_nl, name)
            if similarity.ratio() >= 0.7:
                if sentences_number < 1000:
                    name_dict[last_nl] += 5 
                    name_dict[name] += 5
                else:
                    x = name_dict[last_nl]
                    y = name_dict[name]
                    name_dict[last_nl] += y
                    name_dict[name] += x
    return name_dict



def get_charaters_nltk(book):
    nltk_name_dict = {}
    tokenized_sentences = nltk.sent_tokenize(book)
    for sent in tokenized_sentences:
        chunked = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent)))   
        for item in chunked:
            if type(item) == nltk.Tree and item.label() == "PERSON":
                if " ".join([token for token, pos in item.leaves()]) not in nltk_name_dict:
                    nltk_name_dict[" ".join([token for token, pos in item.leaves()])] = 0
                nltk_name_dict[" ".join([token for token, pos in item.leaves()])] += 1

    return nltk_name_dict, len(tokenized_sentences)



def get_characters_spacy(book):
    spacy_name_dict = {}
    doc = nlp(book)

    for ent in doc.ents:
        if ent.label_ == "PERSON":
            if ent.text not in spacy_name_dict:
                spacy_name_dict[ent.text] = 0
            spacy_name_dict[ent.text] += 1
    
    return spacy_name_dict




ncl = get_characters(open(file_paths[5]).read(), file_paths[5])
print("__________________________________________________________\n",ncl)

In [ ]:
gender_guesser = ggd.Detector()

ns_splitted = []

lista_gg = []

for n in ncl:
    if re.match(".+\s.+", n):
        split = n.split(" ")
        ns_splitted.append(split)
    else:
        ns_splitted.append([n])

for ns in ns_splitted:
    for nns in ns:
        lista_gg.append((ns, nns, gender_guesser.get_gender(nns)))

print("_______GG_________",lista_gg)

In [2]:
from gender_detector import gender_detector as gd
detector = gd.GenderDetector('uk')


lista_gd = []


for ns in ns_splitted:
    for nns in ns:
        lista_gd.append((ns, nns, detector.guess(nns)))

print("___________GD_____________",lista_gd)

NameError: name 'ns_splitted' is not defined